In [1]:
!pip install -r /kaggle/input/m/jbai2025/depthanythingv2/transformers/default/1/Depth-Anything-V2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.8 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import os, gc
import cv2
import sys
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import shutil

#模型加载
sys.path.append("/kaggle/input/m/jbai2025/depthanythingv2/transformers/default/1/Depth-Anything-V2")

from depth_anything_v2.dpt import DepthAnythingV2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DepthAnythingV2(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])
model.load_state_dict(torch.load(
    '/kaggle/input/m/jbai2025/depthanythingv2/transformers/default/1/Depth-Anything-V2/checkpoints/depth_anything_v2_vitl.pth',
    map_location=device
))
model.to(device)
model.eval();

#函数
def save_visualization(rgb, gt, pred, mask, filename):
    # 只考虑有效像素
    valid = mask & (gt > 0) & np.isfinite(gt) & np.isfinite(pred)
    all_depths = np.concatenate([gt[valid], pred[valid]])
    vmin = np.percentile(all_depths, 2)
    vmax = np.percentile(all_depths, 98)

    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    axes[0].imshow(rgb)
    axes[0].set_title("Input RGB Image")
    axes[0].axis("off")

    # ground truth
    im1 = axes[1].imshow(np.where(mask, gt, np.nan), cmap='viridis', vmin=vmin, vmax=vmax)
    axes[1].set_title("Ground Truth Depth (GT)")
    axes[1].axis("off")
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    # prediction
    im2 = axes[2].imshow(np.where(mask, pred, np.nan), cmap='viridis', vmin=vmin, vmax=vmax)
    axes[2].set_title("Predicted Depth (Aligned)")
    axes[2].axis("off")
    plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def scale_match(pred, gt, mask):
    pred = pred[mask]
    gt = gt[mask]
    return np.sum(gt * pred) / np.sum(pred ** 2)

def compute_abs_rel(pred, gt, mask):
    # mask和gt都为True且gt>0的才算有效
    valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan  # 没有有效像素时返回nan，防止出错
    return np.mean(np.abs(pred - gt) / gt)


def compute_delta1(pred, gt, mask=None):
    if mask is None:
        mask = (gt > 0)
    pred, gt = pred[mask], gt[mask]
    epsilon = 1e-6
    thresh = np.maximum(pred / (gt + epsilon), gt / (pred + epsilon))
    return np.mean(thresh < 1.25)

# 路径配置
root_scene_path = '/kaggle/input/diode-val/val/outdoor/scene_00024'
output_root = '/kaggle/working'
os.makedirs(output_root, exist_ok=True)

# 遍历所有 scan 子目录
scan_dirs = sorted(d for d in os.listdir(root_scene_path) if d.startswith('scan_'))

for scan_name in scan_dirs:
    scan_path = os.path.join(root_scene_path, scan_name)
    if not os.path.isdir(scan_path):
        print(f"跳过非目录: {scan_name}")
        continue

    print(f"\n正在处理: {scan_name}")

    vis_folder = os.path.join(output_root, scan_name, 'visualizations')
    os.makedirs(vis_folder, exist_ok=True)

    csv_path = os.path.join(output_root, f"{scan_name}_results.csv")
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)

    results = []

    for img_file in sorted(os.listdir(scan_path)):
        if not img_file.endswith('.png'):
            continue

        try:
            rgb_path = os.path.join(scan_path, img_file)
            gt_path = rgb_path.replace('.png', '_depth.npy')
            mask_path = rgb_path.replace('.png', '_depth_mask.npy')

            if not (os.path.exists(gt_path) and os.path.exists(mask_path)):
                print(f"跳过无 GT/mask 的图像: {img_file}")
                continue

            rgb = cv2.imread(rgb_path)
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
            gt_depth = np.load(gt_path).astype(np.float32)
            if gt_depth.ndim == 3:
                gt_depth = gt_depth[:, :, 0]
            mask = np.load(mask_path).astype(bool)

            with torch.no_grad():
                pred_depth = model.infer_image(rgb)

            if pred_depth.shape != gt_depth.shape:
                pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

            scale = scale_match(pred_depth, gt_depth, mask)
            pred_depth_aligned = pred_depth * scale

            abs_rel = compute_abs_rel(pred_depth_aligned, gt_depth, mask)
            delta1 = compute_delta1(pred_depth_aligned, gt_depth, mask)

            print(f"[{img_file}] AbsRel: {abs_rel:.4f}, δ1: {delta1*100:.2f}%")
            results.append([img_file, abs_rel, delta1])

            vis_filename = os.path.join(vis_folder, img_file.replace('.png', '_vis.png'))
            save_visualization(rgb, gt_depth, pred_depth_aligned, mask, vis_filename)

            del rgb, gt_depth, mask, pred_depth, pred_depth_aligned

        except Exception as e:
            print(f"处理失败: {img_file}，错误: {e}")
            continue

    # 保存当前 scan 的结果 CSV
    df = pd.DataFrame(results, columns=['image', 'AbsRel', 'Delta1'])
    df.to_csv(csv_path, index=False)
    print(f"结果已保存到: {csv_path}")

    # 压缩可视化图
    zip_path = os.path.join(output_root, f'{scan_name}_visualizations')
    shutil.make_archive(zip_path, 'zip', vis_folder)
    print(f"可视化图压缩包保存至: {zip_path}.zip")

    gc.collect()
    torch.cuda.empty_cache()



正在处理: scan_00201
[00024_00201_outdoor_000_000.png] AbsRel: 0.7010, δ1: 13.52%
[00024_00201_outdoor_000_020.png] AbsRel: 1.2441, δ1: 14.72%
[00024_00201_outdoor_010_010.png] AbsRel: 0.9682, δ1: 10.61%
[00024_00201_outdoor_030_000.png] AbsRel: 0.7150, δ1: 13.52%
[00024_00201_outdoor_050_030.png] AbsRel: 0.8505, δ1: 41.50%
[00024_00201_outdoor_060_000.png] AbsRel: 0.7700, δ1: 12.54%
[00024_00201_outdoor_060_040.png] AbsRel: 0.5104, δ1: 39.46%
[00024_00201_outdoor_070_010.png] AbsRel: 0.9753, δ1: 16.68%
[00024_00201_outdoor_070_030.png] AbsRel: 0.7312, δ1: 12.26%
[00024_00201_outdoor_080_000.png] AbsRel: 0.7682, δ1: 10.57%
[00024_00201_outdoor_080_020.png] AbsRel: 0.6787, δ1: 32.23%
[00024_00201_outdoor_080_040.png] AbsRel: 0.4013, δ1: 32.49%
[00024_00201_outdoor_090_010.png] AbsRel: 0.8854, δ1: 7.83%
[00024_00201_outdoor_090_030.png] AbsRel: 0.3750, δ1: 48.85%
[00024_00201_outdoor_090_050.png] AbsRel: 0.2655, δ1: 48.37%
[00024_00201_outdoor_100_000.png] AbsRel: 0.7816, δ1: 9.71%
[00024_0

In [12]:
import os
import pandas as pd

output_dir = '/kaggle/working'
csv_files = []

# 遍历所有目录和子目录，找到 *_results.csv 文件
for root, dirs, files in os.walk(output_dir):
    for file in files:
        if file.endswith('_results.csv'):
            csv_files.append(os.path.join(root, file))

all_dfs = []
summary = []

for f in csv_files:
    df = pd.read_csv(f)
    all_dfs.append(df)
    summary.append({
        'FileName': os.path.basename(f),
        'NumSamples': len(df),
        'MeanAbsRel': df['AbsRel'].mean(),
        'MeanDelta1': df['Delta1'].mean()
    })

if all_dfs:
    df_all = pd.concat(all_dfs, ignore_index=True)
    total = {
        'FileName': 'ALL',
        'NumSamples': len(df_all),
        'MeanAbsRel': df_all['AbsRel'].mean(),
        'MeanDelta1': df_all['Delta1'].mean()
    }
    summary.append(total)
    mean_df = pd.DataFrame(summary)
    mean_df.to_csv(os.path.join(output_dir, "mean.csv"), index=False)
    print(mean_df)
    print(f"所有scan子文件统计均值已保存到: {os.path.join(output_dir, 'mean.csv')}")
else:
    print("未找到任何*_results.csv结果文件！")


                  FileName  NumSamples  MeanAbsRel  MeanDelta1
0   scan_00186_results.csv          48    0.602672    0.189024
1   scan_00200_results.csv          40    0.690656    0.156617
2   scan_00199_results.csv          38    0.688911    0.163458
3   scan_00194_results.csv          44    0.994877    0.213746
4   scan_00184_results.csv          27    0.547667    0.206421
5   scan_00201_results.csv          43    0.676706    0.244545
6   scan_00190_results.csv          25    0.580041    0.205392
7   scan_00187_results.csv          33    0.497098    0.234532
8   scan_00202_results.csv          37    0.770543    0.178150
9   scan_00197_results.csv          61    0.759367    0.172235
10  scan_00195_results.csv          52    0.539056    0.246046
11  scan_00189_results.csv          22    0.587773    0.193171
12  scan_00188_results.csv          25    0.544665    0.208027
13  scan_00193_results.csv          51    0.830306    0.219708
14  scan_00191_results.csv          18    0.580477    0